In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline


class Config:
    def __init__(self):
        pass


def set_config(config_dict):
    config = Config()
    config.__dict__ = config_dict
    return config


In [2]:
config = set_config({
    'row_start': 7228,
    'row_end': 7288,
    "timesteps": 2,
    # "max_batch_size": 13,
    "max_batch_size": 1,
    "layer_size": 1,
    "unit_size": 1,
    "dropout": 0,
    "learning_rate": 0.1,
    "max_epochs": 1,
    "time_col": 'tanggal',
    "feature": ['rr']
})

# usecols = [*[config.time_col], *config.feature]


def train_test_split(dataset, time_step=1):
    dataX, dataY = [], []
    # for i in range(len(dataset)-time_step-1):
    for i in range(len(dataset)-time_step):
        dataX.append(dataset[i:(i+time_step), 0])
        dataY.append(dataset[i+time_step, 0])
    return np.array(dataX), np.array(dataY)


In [3]:
dataset = pd.read_csv('../Data/1985-2021.csv').replace(to_replace=[8888, 9999, 2555], value=np.nan)[config.row_start:config.row_end]
dataset.interpolate(inplace=True)

datelist = [datetime.strptime(date, '%Y-%m-%d').date() for date in list(dataset[config.time_col])]

In [4]:
featureset = dataset[config.feature]
vector_featureset = featureset.values

scaller = MinMaxScaler()
vector_featureset_scaled = scaller.fit_transform(vector_featureset)

train_size = int(vector_featureset_scaled.size * 0.9)
trainset, testset = vector_featureset_scaled[0:train_size], vector_featureset_scaled[train_size:vector_featureset_scaled.size]

X_train, y_train = train_test_split(trainset, time_step=config.timesteps)
# X_test, y_test = train_test_split(testset, time_step=config.timesteps)
# (51, 2)

X_train = np.reshape(X_train, (X_train.shape[0], len(config.feature), X_train.shape[1]))
# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], len(config.feature)))
# X_test = np.reshape(X_test, (X_test.shape[0], len(config.feature), X_test.shape[1]))
# (51, 1, 2)

In [81]:
y_pred_loss = list()
y_true_loss = list()
loss = list()


def mean_squared_error(y_true, y_pred):
    y_pred_loss.append(y_pred)
    y_true_loss.append(y_true)
    loss.append(tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true)))
    return tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true))
    # return tf.math.reduce_mean(tf.square(y_true - y_pred))


def root_mean_squared_error(y_true, y_pred):
    return tf.keras.backend.sqrt(mean_squared_error(y_pred, y_true))


# model = tf.keras.models.Sequential()

# for i in range(0, config.layer_size):
#     model.add(tf.keras.layers.LSTM(
#         units=config.unit_size,
#         return_sequences=False if i == config.layer_size - 1 else True,
#         input_shape=(config.timesteps, 1),
#         go_backwards=True,
#         dropout=config.dropout,
#         weights=[
#             np.float32([[0.5774, 0.5774, 0.5774, 0.5774]]),
#             np.float32([[0.5774, 0.5774, 0.5774, 0.5774]]),
#             np.float32([0, 0, 0, 0])
#         ],
#     ))
# else:
#     # model.compile(optimizer=SGD(learning_rate=config.learning_rate), loss=root_mean_squared_error)
#     print(model.get_weights())
#     model.compile(optimizer=SGD(learning_rate=config.learning_rate), loss='mean_squared_error')
#     model.summary()


model = tf.keras.models.Sequential()

model.add(tf.keras.layers.LSTM(
    units=config.unit_size,
    batch_input_shape=(config.max_batch_size, len(config.feature), config.timesteps),
    go_backwards=True,
    dropout=config.dropout,
    weights=[
        np.array([
            [0.5774, 0.5774, 0.5774, 0.5774],
            [0.5774, 0.5774, 0.5774, 0.5774]
        ]),
        np.array([
            [0.5774, 0.5774, 0.5774, 0.5774]
        ]),
        np.zeros([4])
    ],
))

model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=config.learning_rate),
    loss=mean_squared_error,
    run_eagerly=True
)

model.summary()


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (1, 1)                    16        
                                                                 
Total params: 16
Trainable params: 16
Non-trainable params: 0
_________________________________________________________________


In [82]:
# from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard

batch_loss = list()


history = model.fit(
    X_train,
    y_train,
    shuffle=False,
    epochs=config.max_epochs,
    verbose=1,
    batch_size=config.max_batch_size,
    callbacks=[
        tf.keras.callbacks.LambdaCallback(
            on_epoch_begin=None,
            on_epoch_end=None,
            on_batch_begin=None,
            on_batch_end=lambda batch, logs=None: batch_loss.append(logs['loss']),
            on_train_begin=None,
            on_train_end=None,
        )
    ]
)


52/52 [==============================] - 2s 37ms/step - loss: 0.0797


In [83]:
batch_loss

[0.9924630522727966,
 0.5416995882987976,
 0.3870604932308197,
 0.2904958426952362,
 0.23416030406951904,
 0.19666725397109985,
 0.16890378296375275,
 0.14811253547668457,
 0.13364611566066742,
 0.12047319114208221,
 0.1137012168765068,
 0.10453721880912781,
 0.10512326657772064,
 0.09761984646320343,
 0.09150534123182297,
 0.08656332641839981,
 0.08227597177028656,
 0.07913236320018768,
 0.07658860832452774,
 0.07337412238121033,
 0.07078462839126587,
 0.06784328818321228,
 0.06561852991580963,
 0.06305984407663345,
 0.07602135092020035,
 0.07381964474916458,
 0.07150454819202423,
 0.07906987518072128,
 0.07637372612953186,
 0.07708169519901276,
 0.0757325068116188,
 0.07423459738492966,
 0.07207231223583221,
 0.06997106224298477,
 0.0681360736489296,
 0.08299694210290909,
 0.08258632570505142,
 0.080537348985672,
 0.07876134663820267,
 0.07714874297380447,
 0.0752696841955185,
 0.07869915664196014,
 0.07725026458501816,
 0.07620889693498611,
 0.07453171908855438,
 0.08414769917726517

In [84]:
loss

[<tf.Tensor: shape=(), dtype=float32, numpy=0.99246305>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0909361>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.077782236>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0008019701>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.008818111>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.009202028>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0023229208>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0025738871>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.017914716>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0019168521>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.04598143>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0037332498>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.11215585>,
 <tf.Tensor: shape=(), dtype=float32, numpy=7.548346e-05>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.005902314>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.01243301>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0136783>,
 <tf.Tensor: sha